<a href="https://colab.research.google.com/github/pondpatipatpankaew/Code-Geo/blob/main/Patipat_Pan_sharpening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pan-sharpening**

เทคนิค Pan-sharpening เป็นเทคนิคที่ใช้กับภาพถ่ายดาวเทียม ที่มี Panchromatic band เช่น THEOS-1, LANDSAT-8, LANDSAT-9 เพื่อให้มีรายละเอียดและมีประโยชน์มากขึ้น โดย Lab นี้เราจะทดลองกับข้อมูลดาวเทียม Landsat-9

ข้อมูลถ่ายที่เป็น Multi-Spectral มักจะมีรายละเอียดภาพไม่ค่อยดีนัก เมื่อเทียบกับข้อมูล Panchromatics  ดังนั้น การปรับความคมชัดของภาพช่วยได้โดยการรวมภาพเหล่านี้เข้ากับภาพขาวดำพิเศษที่มีรายละเอียดมากขึ้น ภาพขาวดำที่เรียกว่าภาพแพนโครมาติก มีรายละเอียดในระดับที่สูงกว่า แต่ไม่มีสี เราต้องการใช้รายละเอียดจากภาพแพนโครมาติกและสีจากภาพ Landsat-9 เพื่อสร้างภาพสีที่มีรายละเอียดสูงใหม่

ในการดำเนินการใน Lab นี้ นักศึกษา จำเป็นต้องตรวจสอบให้แน่ใจว่าข้อมูลทั้ง Panchromatic และ Multispectral ของ Landsat-9 อยู่ในตำแหน่งที่สมบูรณ์แบบ เพื่อให้เข้ากันได้อย่างลงตัว

มีหลายวิธีในการรวมภาพเหล่านี้ เช่น Brovey Transform, IHS Transform และ PCA วิธีการเหล่านี้ทำให้แน่ใจว่าภาพใหม่จะคงสีจาก Landsat-9 แต่ยังได้รับรายละเอียดเพิ่มเติมจากภาพแพนโครมาติกด้วย


เมื่อเราทำ Pan-sharpening เสร็จแล้ว เราก็จะได้ภาพใหม่ที่เป็นภาพสีที่มีรายละเอียดสูงขึ้น  โดยมีสีทั้งหมดจาก Landsat-9 และความคมชัดพิเศษจากภาพแบบแพนโครมาติก รูปภาพใหม่นี้สามารถนำไปใช้ได้หลายอย่าง เช่น ศึกษาพื้นดิน ค้นหาการเปลี่ยนแปลง หรือเพียงแค่ดูรายละเอียดเพิ่มเติมเกี่ยวกับโลก ดังนั้น การปรับความคมชัดของภาพเป็นวิธีหนึ่งในการทำให้ภาพจากดาวเทียมมีประโยชน์มากขึ้นสำหรับงานสำคัญทุกประเภท

In [1]:
# ทำการ Authenticate และ initialize Earth Engine
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='ee-pondpatipatpankaew')

In [2]:
# กำหนดพื้นที่สนใจ
geometry = ee.Geometry.Point([98.95799098999555, 18.84423947416328])

# เรียกภาพ L9 ตัวอย่างแล้วดึง RGB และ Pan ออกมา
image = (ee.ImageCollection("LANDSAT/LC09/C02/T1_TOA")
         .filterDate('2022-01-01', '2022-03-30')
         .filterBounds(geometry)
         .sort('CLOUD_COVER')
         .first())

In [3]:
# ทำการ Pan-Sharp
rgb = image.select('B4', 'B3', 'B2')
pan = image.select('B8')

# แปลงเป็น HSV, สลับในแถบ PAN และแปลงกลับเป็น RGB
huesat = rgb.rgbToHsv().select('hue', 'saturation')
upres = ee.Image.cat([huesat, pan]).hsvToRgb()

In [4]:
# สร้างแผนที่
Map = geemap.Map(center=[18.84423947416328, 98.95799098999555], zoom=14)

# แสดง เลเยอร์ ก่อนและหลังโดยใช้พารามิเตอร์ vis เดียวกัน
Map.addLayer(rgb, {'max': 0.28}, 'Original')
Map.addLayer(pan, {'max': 0.28}, 'Pan')
Map.addLayer(upres, {'max': 0.28}, 'Pansharpened')
Map


Map(center=[18.84423947416328, 98.95799098999555], controls=(WidgetControl(options=['position', 'transparent_b…

1. อะไรคือเป้าหมายหลักของการปรับความคมชัดของภาพในการสำรวจระยะไกลโดยเฉพาะเมื่อใช้ภาพ Landsat-9 อธิบายว่าเหตุใดจึงมีความสำคัญในการประมวลผลภาพ

**ตอบ** มีเป้าหมายเพื่อเพิ่มความละเอียดเชิงพื้นที่ของภาพหลายสเปกตรัม โดยอาศัยภาพแพนโครมาติกที่มีความละเอียดสูงกว่า ช่วยให้ได้ภาพสีที่คมชัดขึ้นโดยยังคงรักษาข้อมูลเชิงสเปกตรัมไว้ โดยช่วยในการปรับความคมชัดของภาพอยู่ที่การช่วยเพิ่มความสามารถในการตีความภาพด้วยสายตา ทำให้สามารถแยกแยะลักษณะภูมิประเทศ สิ่งปลูกสร้าง พื้นที่เกษตรกรรม และโครงสร้างพื้นฐานได้ชัดเจนขึ้น

2. อธิบายขั้นตอนสำคัญที่เกี่ยวข้องกับการปรับความคมชัดด้วนเทคนิค Pan-sharpening ตั้งแต่การรับข้อมูลไปจนถึงการสร้างภาพที่ปรับความคมชัด เทคนิค Pan-sharpening มี กระบวนการสุ่มตัวอย่างใหม่ช่วยจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกอย่างไร

**ตอบ** เริ่มจากการคัดเลือกและเตรียมข้อมูลภาพ จากนั้นจัดแนวภาพหลายสเปกตรัมและภาพแพนโครมาติกให้สอดคล้องกันผ่านกระบวนการสุ่มตัวอย่างใหม่ ก่อนทำการผสานข้อมูลโดยแทนค่าความสว่างของภาพสีด้วยค่าจากภาพแพนโครมาติก และแปลงกลับเป็นภาพสี ผลลัพธ์คือภาพที่มีความคมชัดเชิงพื้นที่สูงขึ้นและเหมาะสมต่อการนำไปใช้ในการวิเคราะห์เชิงสำรวจระยะไกล